# 思路
先把transformers官方版本复现一遍

In [1]:
from transformers.trainer_utils import nested_concat, nested_numpify, EvalPrediction
from typing import Dict, List, Optional, Tuple

import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from transformers.data.data_collator import default_data_collator
from loguru import logger

import os
from dataclasses import dataclass
from enum import Enum
from typing import List, Optional, Union

from filelock import FileLock
from code.config import args
import pandas as pd
import numpy as np

from torch.nn import CrossEntropyLoss, MSELoss

from transformers.modeling_outputs import TokenClassifierOutput
# from transformers import PreTrainedTokenizer#, is_torch_available

from transformers import Trainer, AutoModelForTokenClassification, AutoConfig
from transformers.modeling_bert import BertPreTrainedModel, BertModel
from datetime import datetime

from seqeval.metrics import f1_score, precision_score, accuracy_score, recall_score, classification_report

# api

## config

In [2]:
max_seq_length = 250#450
lr = 1e-5

epochs = 10
hidden_dropout_prob=0.1#2

## data prepare

### _cut

In [3]:
def _cut(sentence):
    """
    将一段文本切分成多个句子
    :param sentence:
    :return:
    """
    new_sentence = []
    sen = []
    for i in sentence:
        if i in ['。', '！', '？', '?','；',';'] and len(sen) != 0:
            sen.append(i)
            new_sentence.append("".join(sen))
            sen = []
            continue
        sen.append(i)

    if len(new_sentence) <= 1: # 一句话超过max_seq_length且没有句号的，用","分割，再长的不考虑了。
        new_sentence = []
        sen = []
        for i in sentence:
            if i.split(' ')[0] in ['，', ','] and len(sen) != 0:
                sen.append(i)
                new_sentence.append("".join(sen))
                sen = []
                continue
            sen.append(i)
    if len(sen) > 0:  # 若最后一句话无结尾标点，则加入这句话
        new_sentence.append("".join(sen))
    return new_sentence

def cut_test_set(text_list,len_treshold):
    """
    
    :text_list: 文本列表
    :return
    
    """
    cut_text_list = []
    cut_index_list = []
    for text in text_list:

        temp_cut_text_list = []
        text_agg = ''
        if len(text) < len_treshold:
            temp_cut_text_list.append(text)
        else:
            sentence_list = _cut(text)  # 一条数据被切分成多句话
            for sentence in sentence_list:
                if len(text_agg) + len(sentence) < len_treshold:
                    text_agg += sentence
                else:
                    temp_cut_text_list.append(text_agg)
                    text_agg = sentence
            temp_cut_text_list.append(text_agg)  # 加上最后一个句子

        cut_index_list.append(len(temp_cut_text_list))
        cut_text_list += temp_cut_text_list

    return cut_text_list, cut_index_list

In [4]:
test1 = """  1.忌食生冷食物；2.患有其他疾病患者，应在医师指导下服用；3.平素月经正常，突然出现月经过少，或经期错后，应去医院就诊；4.治疗痛经，宜在经前3-5天开始服药，连服1周；5.如有生育要求，服药期间应避孕，或月经来潮后在医师指导下服用；6.服药后痛经不减轻，或重度痛经者，应到医院就诊；7.青春期少女及更年期妇女应在医师指导下服用；8.服药1个月症状去缓解，应去医院就诊；9.对本品过敏者禁用，过敏体质者慎用；10.本品性状发生改变时禁止使用；11.请将本品放在儿童不能接触的地方；12.如正在使用其他药品，使用本品请咨询医师或药师。 269
用于月经量少 非处方药物（乙类）,国家基本药物目录（2012）  1、忌食生冷食物；2、气血两虚引起的月经量少，色淡质稀，伴有头晕心悸，疲乏无力等不宜选用本药；3、有高血压、心脏病、肾病、糖尿病或正在接受其他治疗的患者均应在医师指导下服用；4、平素月经量正常，突然出现经量少，须去医院就诊；5、青春期少女及更年期妇女应在医师指导下服药；6、各种流产后腹痛伴有阴道出血，服药一周无效者应去医院就诊；7、按照用法用量服用，服药过程中出现不良反应应停药，并向医师咨询；8、对本品过敏者禁用，过敏体质者慎用；9、本品性状发生改变时禁止使用；10、请将本品放在儿童不能接触的地方；11、如正在使用其他药品，使用本品前请咨询医师或药师。"""
test1 = """ 1.忌食生冷及油腻难消化的食品2.服药期间保持情绪乐观切忌生气恼怒3.肝肾阴虚气滞不运所致的胸胁疼痛胸腹胀满咽喉干燥舌无津液者慎用4.火郁证者不适用主要表现为口苦咽干面色红赤心中烦热胁胀不眠大便秘结5.有高血压心脏病肝病糖尿病肾病等慢性病严重者应在医师指导下服用6.服药天症状无缓解应去医院就诊7.儿童年老体弱者应在医师指导下服用8.对本品过敏者禁用过敏体质者慎用9.药品性状发生改变时禁止使用10.儿童必须在成人监护下使用11.请将此药品放在儿童不能接触的地方12.如正在服用其它药品使用吉春靓美人 红花逍遥片前请咨询医师或药师  吉林吉春制药股份有限公司  药用铝塑泡罩包装，12片/板*3板/盒  尚不明确。"""
# test1 = " 灌肠用。取本品50ml，将药液加温至38~39°C，臀部抬高10cm插管，肛管插入深度10~15cm。肛管插入后，讲管端套的熟料瓶颈部，加压挤入即可。灌入后膝胸卧位30分钟。每日一次，两周为一个疗程。月经干净后3~5天开始用药。  红虎灌肠液（50毫升装）-安徽天洋药业  清热解毒，化湿除带，祛瘀止痛，散结消癥，用于慢性盆腔炎所致小腹疼痛，腰骶酸痛，带下量多，或有发热 安徽天洋药业有限公司 "

In [5]:
len(test1)

308

In [6]:
sens, num = cut_test_set([test1], 250)

In [7]:
print(sens, num )

['', ' 1.忌食生冷及油腻难消化的食品2.服药期间保持情绪乐观切忌生气恼怒3.肝肾阴虚气滞不运所致的胸胁疼痛胸腹胀满咽喉干燥舌无津液者慎用4.火郁证者不适用主要表现为口苦咽干面色红赤心中烦热胁胀不眠大便秘结5.有高血压心脏病肝病糖尿病肾病等慢性病严重者应在医师指导下服用6.服药天症状无缓解应去医院就诊7.儿童年老体弱者应在医师指导下服用8.对本品过敏者禁用过敏体质者慎用9.药品性状发生改变时禁止使用10.儿童必须在成人监护下使用11.请将此药品放在儿童不能接触的地方12.如正在服用其它药品使用吉春靓美人 红花逍遥片前请咨询医师或药师  吉林吉春制药股份有限公司  药用铝塑泡罩包装，', '12片/板*3板/盒  尚不明确。'] [3]


In [8]:
len(sens[2])

17

In [9]:
for s in cut_test_set([test1], 50)[0]:
    print(s)
    print(len(s))


0
 1.忌食生冷及油腻难消化的食品2.服药期间保持情绪乐观切忌生气恼怒3.肝肾阴虚气滞不运所致的胸胁疼痛胸腹胀满咽喉干燥舌无津液者慎用4.火郁证者不适用主要表现为口苦咽干面色红赤心中烦热胁胀不眠大便秘结5.有高血压心脏病肝病糖尿病肾病等慢性病严重者应在医师指导下服用6.服药天症状无缓解应去医院就诊7.儿童年老体弱者应在医师指导下服用8.对本品过敏者禁用过敏体质者慎用9.药品性状发生改变时禁止使用10.儿童必须在成人监护下使用11.请将此药品放在儿童不能接触的地方12.如正在服用其它药品使用吉春靓美人 红花逍遥片前请咨询医师或药师  吉林吉春制药股份有限公司  药用铝塑泡罩包装，
291
12片/板*3板/盒  尚不明确。
17


### InputExample | InputFeatures | Split

In [10]:
@dataclass
class InputExample:
    """
    A single training/test example for token classification.
    Args:
        guid: Unique id for the example.
        words: list. The words of the sequence.
        labels: (Optional) list. The labels for each word of the sequence. This should be
        specified for train and dev examples, but not for test examples.
    """

    guid: str
    words: List[str]
    labels: Optional[List[str]]


@dataclass
class InputFeatures:
    """
    A single set of features of data.
    Property names are the same names as the corresponding inputs to a model.
    """

    input_ids: List[int]
    attention_mask: List[int]
    token_type_ids: Optional[List[int]] = None
    label_ids: Optional[List[int]] = None
    input_len: Optional[int] = None

### read_examples_from_file

In [11]:
def read_examples_from_file(fp) -> List[InputExample]:
    """
    :fp: corpus路径
    """
    file_path = os.path.join(fp)
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid=f"{guid_index}", words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid=f"{mode}-{guid_index}", words=words, labels=labels))
    return examples

In [12]:
# examples = read_examples_from_file(f'{args.DATA_GEN}train_val.txt')
# examples[:2]

### read_examples_from_raw

In [13]:
def get_examples(content, text_length, mark, i):
    examples_ = []
    cut_text_list, cut_index_list = cut_test_set([content],text_length)
#         print(cut_text_list, cut_index_list)
    begin = 0
#     contents
    for ii in range(cut_index_list[0]):
        words = list(cut_text_list[ii])
#             if(len(words)>text_length):
#                 print(cut_text_list[ii], len(words))
#                 break
        labels = mark[begin: (begin+len(words))]
        begin += len(words)
        if words:
            examples_.append(InputExample(guid=f"{i}-{ii}", words=words, labels=labels))
    return examples_

In [14]:
def read_examples_from_raw(fp, indexes, text_length, enforce=False) -> List[InputExample]:
    """
    :fp: corpus路径
    """
    examples = []
    for i in indexes:#range(1000):
        with open(os.path.join(fp, f'{i}.txt')) as f:
            content = f.read()
#             print(content)
        mark = ['O']*len(content)
        with open(os.path.join(fp, f'{i}.ann')) as f:
            for line in f:
                line = line.strip()
                if line.strip():
                    items, name = line.split('\t')[1:]
                    cat, start, end = items.split(' ')
                    start, end = int(start), int(end)
                    mark[start] = f'B-{cat}'
                    for ii in range((start+1), end):
                        mark[ii] = f'I-{cat}'
#                     print(name, content[start: end])
#         print(mark)
        examples_ =  get_examples(content, text_length, mark, i)
        examples.extend(examples_)
        if enforce:
            # 数据增强
            if len(content) > text_length * 1.5:
                begin_ = content.find('。')
                content_ = content[(begin_+1):]
                if len(content_) > text_length // 2:
                    examples_ =  get_examples(content_, text_length, mark[(begin_+1):], i)
                    examples.extend(examples_)
                end_ = content.rfind('。')
                content_ = content[:(end_+1)]
                if len(content_) > text_length // 2:
                    examples_ =  get_examples(content_, text_length, mark[:(end_+1)], i)
                    examples.extend(examples_)
    return examples

In [15]:
examples = read_examples_from_raw(args.DATA_DIR+'train/', [2, 3,7, 8,9], max_seq_length)
examples[:2]

[InputExample(guid='2-0', words=[' ', '湖', '北', '御', '金', '丹', '药', '业', '有', '限', '公', '司', ' ', ' ', '非', '处', '方', '药', '物', '（', '乙', '类', '）', ',', '国', '家', '基', '本', '药', '物', '目', '录', '（', '2', '0', '1', '2', '）', ' ', ' ', '密', '封', '(', '1', '0', '～', '3', '0', '℃', ')', '。', ' ', ' ', '每', '袋', '装', '1', '5', '克', ' ', ' ', '孕', '妇', '禁', '用', '。', ' ', ' ', '尚', '不', '明', '确', '。', ' ', ' ', '孕', '妇', '禁', '用', '。', ' ', ' ', '活', '血', '调', '经', '。', '用', '于', '血', '瘀', '所', '致', '的', '月', '经', '不', '调', '，', '症', '见', '经', '水', '量', '少', '。', '用', '于', '月', '经', '量', '少', '，', '月', '经', '后', '错', '，', '经', '来', '腹', '痛', '月', '经', '不', '调', ' ', '开', '水', '冲', '服', '。', '一', '次', '1', '5', '克', '，', '一', '日', '2', '次', '。', ' ', ' ', '祛', '瘀', '生', '新', '。', ' '], labels=['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [16]:
len(examples)

11

### read_examples_from_testdir

In [17]:
def read_examples_from_testdir(fp, text_length) -> List[InputExample]:
    """
    :fp: corpus路径
    """
    test_files = os.listdir(fp)
    examples = []
    for file in test_files:
        guid_index = int(file.split('.')[0])
        with open(os.path.join(fp, file), encoding="utf-8") as f:
            for content_str in f:
                cut_text_list, cut_index_list = cut_test_set([content_str],text_length)
                for i in range(cut_index_list[0]):
                    words = list(cut_text_list[i])
                    if(len(words)>text_length):
                        print(words, len(words))
                    labels = ['O'] * len(words)
                    examples.append(InputExample(guid=f"{guid_index}-{i+1}", words=words, labels=labels))
            
    return examples

In [18]:
examples_test = read_examples_from_testdir(f'{args.DATA_DIR}test/', max_seq_length)
examples_test[:2]

[InputExample(guid='1469-1', words=[' ', '孕', '妇', '忌', '服', '。', ' ', ' ', '养', '血', '化', '瘀', '，', '舒', '肝', '调', '经', '。', '用', '于', '气', '滞', '血', '瘀', '所', '致', '的', '月', '经', '不', '调', '，', '痛', '经', '，', '症', '见', '月', '经', '量', '少', '有', '血', '块', '、', '经', '行', '后', '期', '或', '前', '后', '不', '定', '、', '经', '行', '小', '腹', '胀', '痛', '孕', '妇', ' ', '非', '处', '方', '药', '物', '（', '甲', '类', '）', ' ', ' ', '口', '服', '，', '一', '次', '1', '丸', '，', '一', '日', '2', '次', '。', ' ', ' ', '每', '丸', '重', '9', 'g', ' ', ' ', '天', '津', '中', '新', '药', '业', '集', '团', '股', '份', '有', '限', '公', '司', '达', '仁', '堂', '制', '药', '厂', ' ', ' ', '孕', '妇', '忌', '服', '。', ' '], labels=['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [19]:
len(examples_test)

851

### get_labels

In [20]:
def get_labels(path: str) -> List[str]:
    if path:
        with open(path, "r") as f:
            labels = f.read().splitlines()
        if "O" not in labels:
            labels = ["O"] + labels
        return labels
    else:
        return ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

In [21]:
labels = get_labels(f'{args.DATA_GEN}torch_ner_data/labels.txt')
labels

['O',
 'B-DRUG',
 'I-DRUG',
 'B-DRUG_INGREDIENT',
 'I-DRUG_INGREDIENT',
 'B-DISEASE',
 'I-DISEASE',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'B-SYNDROME',
 'I-SYNDROME',
 'B-DISEASE_GROUP',
 'I-DISEASE_GROUP',
 'B-FOOD',
 'I-FOOD',
 'B-FOOD_GROUP',
 'I-FOOD_GROUP',
 'B-PERSON_GROUP',
 'I-PERSON_GROUP',
 'B-DRUG_GROUP',
 'I-DRUG_GROUP',
 'B-DRUG_DOSAGE',
 'I-DRUG_DOSAGE',
 'B-DRUG_TASTE',
 'I-DRUG_TASTE',
 'B-DRUG_EFFICACY',
 'I-DRUG_EFFICACY']

In [22]:
label_map = {i: label for i, label in enumerate(labels)}

In [23]:
label_map

{0: 'O',
 1: 'B-DRUG',
 2: 'I-DRUG',
 3: 'B-DRUG_INGREDIENT',
 4: 'I-DRUG_INGREDIENT',
 5: 'B-DISEASE',
 6: 'I-DISEASE',
 7: 'B-SYMPTOM',
 8: 'I-SYMPTOM',
 9: 'B-SYNDROME',
 10: 'I-SYNDROME',
 11: 'B-DISEASE_GROUP',
 12: 'I-DISEASE_GROUP',
 13: 'B-FOOD',
 14: 'I-FOOD',
 15: 'B-FOOD_GROUP',
 16: 'I-FOOD_GROUP',
 17: 'B-PERSON_GROUP',
 18: 'I-PERSON_GROUP',
 19: 'B-DRUG_GROUP',
 20: 'I-DRUG_GROUP',
 21: 'B-DRUG_DOSAGE',
 22: 'I-DRUG_DOSAGE',
 23: 'B-DRUG_TASTE',
 24: 'I-DRUG_TASTE',
 25: 'B-DRUG_EFFICACY',
 26: 'I-DRUG_EFFICACY'}

### convert_examples_to_features

In [24]:
def convert_examples_to_features(
    examples: List[InputExample],
    label_list: List[str],
    max_seq_length: int,
    tokenizer,#: PreTrainedTokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=0,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
) -> List[InputFeatures]:
    """ Loads a data file into a list of `InputFeatures`
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """
    # TODO clean up all this to leverage built-in features of tokenizers

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10_000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        input_len = len(example.words)
        for word, label in zip(example.words, example.labels):
            if not word.isspace():
                word_tokens = tokenizer.tokenize(word)
            else:
                word_tokens = word
            # bert-base-multilingual-cased sometimes output "nothing ([]) when calling tokenize with just a space.
            if len(word_tokens) > 0:
                tokens.extend(word_tokens)
                # Use the real label id for the first token of the word, and padding ids for the remaining tokens
                label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = tokenizer.num_special_tokens_to_add()
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

#         if ex_index < 5:
#             logger.info("*** Example ***")
#             logger.info("guid: %s", example.guid)
#             logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
#             logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
#             logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
#             logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
#             logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        if "token_type_ids" not in tokenizer.model_input_names:
            segment_ids = None

            
            
        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=input_mask, token_type_ids=segment_ids, 
                label_ids=label_ids, input_len=input_len
            )
        )
    return features

In [25]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# features = convert_examples_to_features(examples, labels, 250, tokenizer)
# features[1]

### NerDataset

In [26]:
class NerDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    features: List[InputFeatures]
    pad_token_label_id: int = nn.CrossEntropyLoss().ignore_index
#     pad_token_label_id: int = 0
    # Use cross entropy ignore_index as padding label id so that only
    # real label ids contribute to the loss later.

    def __init__(
        self,
        examples,
        tokenizer, #: PreTrainedTokenizer,
        labels: List[str],
        model_type: str,
        max_seq_length: Optional[int] = None,
    ):
        
        # TODO clean up all this to leverage built-in features of tokenizers
        self.features = convert_examples_to_features(
            examples,
            labels,
            max_seq_length,
            tokenizer,
            cls_token_at_end=bool(model_type in ["xlnet"]),
            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=2 if model_type in ["xlnet"] else 0,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,
            # roberta uses an extra separator b/w pairs of sentences, cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
            pad_on_left=bool(tokenizer.padding_side == "left"),
            pad_token=tokenizer.pad_token_id,
            pad_token_segment_id=tokenizer.pad_token_type_id,
            pad_token_label_id=self.pad_token_label_id,
        )

    def __len__(self):
        return len(self.features)

    def __getitem__(self, i) -> InputFeatures:
        return self.features[i]


### data loader

In [27]:
# train_data_loader = DataLoader(features, batch_size=6, shuffle=True,collate_fn=default_data_collator)

# len(train_data_loader)  # size of features / batch_size

In [28]:
# for step, inputs in enumerate(train_data_loader):
#     print(step)
#     print(inputs)
#     break

## Trainer

### align_predictions

In [29]:
def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(label_map[label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])
#         print(len(out_label_list[i]), len(preds_list[i]))
    return preds_list, out_label_list


In [30]:
def compute_metrics(preds_list, out_label_list) -> Dict:
    
#     print(preds_list)
#     print(out_label_list)
    return {
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

### train_args

### MyBertForTokenClassification

In [31]:
class MyBertForTokenClassification(BertPreTrainedModel):

    authorized_unexpected_keys = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
    
        self.bert = BertModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()
    
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=True,
        return_dict=True,
        input_len=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

#         sequence_output = outputs[0]
        sequence_output = (outputs[0]+outputs.hidden_states[-1]+outputs.hidden_states[-2]
                            + outputs.hidden_states[-3]+outputs.hidden_states[-4]
                            +outputs.hidden_states[-5])/6

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
#         return outputs, logits
        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

### MyBertForTokenClassification_CRF

In [32]:
class MyBertForTokenClassification_CRF(BertPreTrainedModel):

    authorized_unexpected_keys = [r"pooler"]

    def __init__(self, config, tag_to_ix):
        super().__init__(config)
        self.num_labels = config.num_labels
    
        self.bert = BertModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.classifier = nn.Linear(config.hidden_size, self.tagset_size)
        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))
        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to('cuda')
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
        
    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to('cuda')
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to('cuda'), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
    
    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to('cuda')
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, feats, tags):
#         feats = self._get_feats(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
    def _get_feats(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=True,
        return_dict=True,
        input_len=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

#         sequence_output = outputs[0]
        sequence_output = (outputs[0]+outputs.hidden_states[-1]+outputs.hidden_states[-2]
                            + outputs.hidden_states[-3]+outputs.hidden_states[-4]
                            +outputs.hidden_states[-5])/6

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        return logits
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=True,
        return_dict=True,
        input_len=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

#         sequence_output = outputs[0]
        sequence_output = (outputs[0]+outputs.hidden_states[-1]+outputs.hidden_states[-2]
                            + outputs.hidden_states[-3]+outputs.hidden_states[-4]
                            +outputs.hidden_states[-5])/6

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
#         return outputs, logits
        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

## 5 fold

In [33]:
from sklearn.model_selection import KFold

folds = KFold(5, shuffle=True, random_state=2019)

In [34]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

In [35]:
from transformers import AutoTokenizer
# model_name_or_path = 'bert-base-cased'
# model_name_or_path = 'hfl/chinese-bert-wwm'
model_name_or_path = 'hfl/chinese-roberta-wwm-ext'
# model_name_or_path = 'hfl/chinese-roberta-wwm-ext-large'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


In [36]:
config = AutoConfig.from_pretrained(
        model_name_or_path,
        num_labels=len(labels),
        id2label= label_map,
        label2id={label: i for i, label in enumerate(labels)},
    )

In [37]:
config.hidden_dropout_prob = hidden_dropout_prob

In [38]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-DRUG",
    "2": "I-DRUG",
    "3": "B-DRUG_INGREDIENT",
    "4": "I-DRUG_INGREDIENT",
    "5": "B-DISEASE",
    "6": "I-DISEASE",
    "7": "B-SYMPTOM",
    "8": "I-SYMPTOM",
    "9": "B-SYNDROME",
    "10": "I-SYNDROME",
    "11": "B-DISEASE_GROUP",
    "12": "I-DISEASE_GROUP",
    "13": "B-FOOD",
    "14": "I-FOOD",
    "15": "B-FOOD_GROUP",
    "16": "I-FOOD_GROUP",
    "17": "B-PERSON_GROUP",
    "18": "I-PERSON_GROUP",
    "19": "B-DRUG_GROUP",
    "20": "I-DRUG_GROUP",
    "21": "B-DRUG_DOSAGE",
    "22": "I-DRUG_DOSAGE",
    "23": "B-DRUG_TASTE",
    "24": "I-DRUG_TASTE",
    "25": "B-DRUG_EFFICACY",
    "26": "I-DRUG_EFFICACY"
  },
  "initializer_range": 0.02,
 

In [39]:
def create_optimizer_and_scheduler(model, num_training_steps: int):
    """
    Setup the optimizer and the learning rate scheduler.

    We provide a reasonable default that works well. If you want to use something else, you can pass a tuple in the
    Trainer's init through :obj:`optimizers`, or subclass and override this method in a subclass.
    """
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#             "weight_decay": self.args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=lr,
    )
#         lr_scheduler = get_linear_schedule_with_warmup(
#             optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=num_training_steps
#         )
    return optimizer

In [40]:
testset = NerDataset(examples_test, tokenizer, labels, config.model_type, max_seq_length+4)
test_data_loader = DataLoader(testset, batch_size=6, shuffle=False,collate_fn=default_data_collator)  

2020-10-13 12:11:30.987 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d


In [41]:
label2id={label: i for i, label in enumerate(labels)}
START_TAG = "<START>"
STOP_TAG = "<STOP>"
label2id[START_TAG] = len(label2id)
label2id[STOP_TAG] = len(label2id)

In [43]:
# model = MyBertForTokenClassification_CRF(config, label2id)

In [51]:
preds_ = None
isInfer = False
epochs = 50
isInfer = True
epochs = 10
for fold_, (trn_idx, val_idx) in enumerate(folds.split(range(1000))):
#     if fold_<2: continue
    print(f'fold{fold_}')
#     print(trn_idx, val_idx)
    examples_train = read_examples_from_raw(args.DATA_DIR+'train/', trn_idx, max_seq_length, enforce=False)#pd.Series(examples).iloc[trn_idx].tolist()
    examples_val = read_examples_from_raw(args.DATA_DIR+'train/', val_idx, max_seq_length, enforce=False)#pd.Series(examples).iloc[val_idx].tolist()
    print(f'trainset num:{len(examples_train)}, valset num: {len(examples_val)}')
    trainset = NerDataset(examples_train, tokenizer, labels, config.model_type, max_seq_length+4)
    train_data_loader = DataLoader(trainset, batch_size=6, shuffle=True,collate_fn=default_data_collator)
    valset = NerDataset(examples_val, tokenizer, labels, config.model_type, max_seq_length+4)
    val_data_loader = DataLoader(valset, batch_size=6, shuffle=False,collate_fn=default_data_collator)
    
#     model = MyBertForTokenClassification.from_pretrained(model_name_or_path, config=config)
#     model = MyBertForTokenClassification.from_pretrained(model_name_or_path, config=config, tag_to_ix=label2id)
    model = MyBertForTokenClassification_CRF(config, label2id)
    model.from_pretrained(model_name_or_path, tag_to_ix=label2id)
    
    model = model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     optimizer = create_optimizer_and_scheduler(model, 0)
    time_start = datetime.now()
    

    bestf1 = 0
    for epoch in range(epochs):
        losst = 0
        model.train()
        for step, inputs in enumerate(train_data_loader):
            optimizer.zero_grad()
        #     print(step)
        #     print(inputs)
            for k, v in inputs.items():
                if isinstance(v, torch.Tensor):
                    inputs[k] = v.to('cuda')
#             print(inputs)
#             break
            feats = model._get_feats(**inputs)
            loss = 0
            for feat, label in zip(feats, inputs['labels']):
                ss = (label!=-100).nonzero().squeeze()
                feat_pure = feat.index_select(0, ss)
                label = label.index_select(0, ss)
                loss = model.neg_log_likelihood(feat_pure, label)
                loss += loss
            losst += loss
            loss.backward()
            optimizer.step()
            if step % 100 == 0:
                print(epoch, step, loss.item())
#                 break
#         break
        with torch.no_grad():
            model.eval()
            lossv = 0
            preds: torch.Tensor = None
            label_ids: torch.Tensor = None
            for stepv, inputsv in enumerate(val_data_loader):
                for k, v in inputsv.items():
                    if isinstance(v, torch.Tensor):
                        inputsv[k] = v.to('cuda')
                        
                
                feats = model._get_feats(**inputsv)
                loss = 0
                for feat, label in zip(feats, inputsv['labels']):
                    ss = (label!=-100).nonzero().squeeze()
                    feat_pure = feat.index_select(0, ss)
                    label = label.index_select(0, ss)
                    loss = model.neg_log_likelihood(feat_pure, label)
                    loss += loss
                lossv += loss
                print(stepv, lossv)
#                 outputs = model(**inputsv)
#                 lossv += outputs[0] 
#                 logits = outputs[1]
#                 preds = logits if preds is None else nested_concat(preds, logits, dim=0)
#                 label_ids = inputsv['labels'] if label_ids is None else nested_concat(label_ids, inputsv['labels'], dim=0)
#             preds = nested_numpify(preds)
#             label_ids = nested_numpify(label_ids)
#             p = EvalPrediction(predictions=preds, label_ids=label_ids)
#             preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
# #             print(preds_list, out_label_list)
# #             break
#             metrics = compute_metrics(preds_list, out_label_list)
#             if metrics['f1'] > bestf1 and isInfer:
# #                 model.save_pretrained(f'prerained-{fold_}-{epoch}')
#                 bestf1 = metrics['f1']
#                 print(f'update bestf1 to {bestf1}')
#                 if epoch > 3:
#                     print(epoch, 'test...')
#                     predst: torch.Tensor = None
#                     label_idst: torch.Tensor = None
#                     for stepv, inputsv in enumerate(test_data_loader):
#                         for k, v in inputsv.items():
#                             if isinstance(v, torch.Tensor):
#                                 inputsv[k] = v.to('cuda')
#                         outputs = model(**inputsv)
#                         logits = outputs[1]
#                         predst = logits if predst is None else nested_concat(predst, logits, dim=0)
#                         label_idst = inputsv['labels'] if label_idst is None else nested_concat(label_idst, inputsv['labels'], dim=0)
#                     predst = nested_numpify(predst)
#                     label_idst = nested_numpify(label_idst)              
            now = datetime.now()
            metrics = None
            print(f'{now.strftime("%Y-%m-%d %H:%M:%S")} {int((now-time_start).total_seconds())}s epoch: {epoch}; train loss: {round(losst.item(),4)}; val loss: {round(lossv.item(),4)}\nmetrics:{metrics}')
#             break
        time_start = now
    break
    preds_ = predst if preds_ is None else preds_+predst
# trainset num:3446, valset num: 371    
#     break

2020-10-13 16:23:15.925 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d


fold0
trainset num:1575, valset num: 371


2020-10-13 16:23:21.183 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d
Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing MyBertForTokenClassification_CRF: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MyBertForTokenClassification_CRF from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing MyBertForTokenClassification_CRF from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

0 0 1153.072265625


KeyboardInterrupt: 

In [44]:
preds_ = None
isInfer = False
epochs = 50
isInfer = True
epochs = 10
for fold_, (trn_idx, val_idx) in enumerate(folds.split(range(1000))):
#     if fold_<2: continue
    print(f'fold{fold_}')
#     print(trn_idx, val_idx)
    examples_train = read_examples_from_raw(args.DATA_DIR+'train/', trn_idx, max_seq_length, enforce=False)#pd.Series(examples).iloc[trn_idx].tolist()
    examples_val = read_examples_from_raw(args.DATA_DIR+'train/', val_idx, max_seq_length, enforce=False)#pd.Series(examples).iloc[val_idx].tolist()
    print(f'trainset num:{len(examples_train)}, valset num: {len(examples_val)}')
    trainset = NerDataset(examples_train, tokenizer, labels, config.model_type, max_seq_length+4)
    train_data_loader = DataLoader(trainset, batch_size=6, shuffle=True,collate_fn=default_data_collator)
    valset = NerDataset(examples_val, tokenizer, labels, config.model_type, max_seq_length+4)
    val_data_loader = DataLoader(valset, batch_size=6, shuffle=False,collate_fn=default_data_collator)
    
#     model = MyBertForTokenClassification.from_pretrained(model_name_or_path, config=config)
#     model = MyBertForTokenClassification.from_pretrained(model_name_or_path, config=config, tag_to_ix=label2id)
    model = MyBertForTokenClassification_CRF(config, label2id)
    model.from_pretrained(model_name_or_path, tag_to_ix=label2id)
    
    model = model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     optimizer = create_optimizer_and_scheduler(model, 0)
    time_start = datetime.now()
    

    bestf1 = 0
    for epoch in range(epochs):
        losst = 0
        model.train()
        for step, inputs in enumerate(train_data_loader):
            optimizer.zero_grad()
        #     print(step)
        #     print(inputs)
            for k, v in inputs.items():
                if isinstance(v, torch.Tensor):
                    inputs[k] = v.to('cuda')
#             print(inputs)
#             break
            feats = model._get_feats(**inputs)
            loss = 0
            for feat, label in zip(feats, inputs['labels']):
                    feat_pure = feat.index_select(0, (label!=-100).nonzero().squeeze()).to('cuda')
                    label = label.index_select(0, (label!=-100).nonzero().squeeze())
                    loss = model.neg_log_likelihood(feat_pure, label)
                    loss += loss
#                     print(model._viterbi_decode(feat_pure))
                    
#             break
#             outputs = model(**inputs)
#             loss = outputs[0]
            losst += loss
            loss.backward()
            optimizer.step()
#             if step % 100 == 0:
#                 print(epoch, step, loss.item())
        break
        with torch.no_grad():
            model.eval()
            lossv = 0
            preds: torch.Tensor = None
            label_ids: torch.Tensor = None
            for stepv, inputsv in enumerate(val_data_loader):
                for k, v in inputsv.items():
                    if isinstance(v, torch.Tensor):
                        inputsv[k] = v.to('cuda')
                outputs = model(**inputsv)
                lossv += outputs[0] 
                logits = outputs[1]
                preds = logits if preds is None else nested_concat(preds, logits, dim=0)
                label_ids = inputsv['labels'] if label_ids is None else nested_concat(label_ids, inputsv['labels'], dim=0)
            preds = nested_numpify(preds)
            label_ids = nested_numpify(label_ids)
            p = EvalPrediction(predictions=preds, label_ids=label_ids)
            preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
#             print(preds_list, out_label_list)
#             break
            metrics = compute_metrics(preds_list, out_label_list)
            if metrics['f1'] > bestf1 and isInfer:
#                 model.save_pretrained(f'prerained-{fold_}-{epoch}')
                bestf1 = metrics['f1']
                print(f'update bestf1 to {bestf1}')
                if epoch > 3:
                    print(epoch, 'test...')
                    predst: torch.Tensor = None
                    label_idst: torch.Tensor = None
                    for stepv, inputsv in enumerate(test_data_loader):
                        for k, v in inputsv.items():
                            if isinstance(v, torch.Tensor):
                                inputsv[k] = v.to('cuda')
                        outputs = model(**inputsv)
                        logits = outputs[1]
                        predst = logits if predst is None else nested_concat(predst, logits, dim=0)
                        label_idst = inputsv['labels'] if label_idst is None else nested_concat(label_idst, inputsv['labels'], dim=0)
                    predst = nested_numpify(predst)
                    label_idst = nested_numpify(label_idst)              
            now = datetime.now()
            print(f'{now.strftime("%Y-%m-%d %H:%M:%S")} {int((now-time_start).total_seconds())}s epoch: {epoch}; train loss: {round(losst.item(),4)}; val loss: {round(lossv.item(),4)}\nmetrics:{metrics}')
#             break
        time_start = now
    break
    preds_ = predst if preds_ is None else preds_+predst
# trainset num:3446, valset num: 371    
#     break

2020-10-13 12:11:47.106 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d


fold0
trainset num:1575, valset num: 371


2020-10-13 12:11:52.342 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d
Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing MyBertForTokenClassification_CRF: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MyBertForTokenClassification_CRF from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing MyBertForTokenClassification_CRF from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

In [56]:
feats.shape

torch.Size([6, 254, 27])

In [80]:
inputs['labels'].shape

torch.Size([6, 254])

In [ ]:
def t(feats_, labels_):
    for feat in feats_:
        
    (inputs['labels']!=-100)[0].nonzero().squeeze()

In [45]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [107]:
model.tagset_size

29

In [123]:
(label!=-100).nonzero().squeeze()

tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
         85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
         99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 1

In [46]:
for feat, label in zip(feats, inputs['labels']):
#     print(feat, label)
    feat_pure = feat.index_select(0, (label!=-100).nonzero().squeeze()).to('cuda')
    label = label.index_select(0, (label!=-100).nonzero().squeeze())
#     print(label)
#     print(feat_pure.shape)
#     print(model._forward_alg(feat_pure))
#     print(model._score_sentence(feat_pure, label))
    print(model.neg_log_likelihood(feat_pure, label))
    print(model._viterbi_decode(feat_pure))
    break

<ipython-input-46-ec665b652d09>:3: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  feat_pure = feat.index_select(0, (label!=-100).nonzero().squeeze()).to('cuda')


tensor([834.6583], device='cuda:0', grad_fn=<SubBackward0>)
(tensor(604.1512, device='cuda:0', grad_fn=<SelectBackward>), [0, 12, 14, 21, 26, 0, 12, 17, 17, 17, 17, 7, 17, 7, 17, 0, 12, 2, 12, 22, 24, 14, 21, 7, 17, 7, 17, 17, 8, 26, 0, 12, 22, 24, 14, 21, 20, 8, 26, 0, 12, 2, 12, 17, 19, 15, 18, 12, 17, 17, 19, 0, 12, 22, 24, 14, 21, 2, 10, 24, 14, 8, 26, 0, 12, 22, 24, 14, 21, 20, 26, 0, 12, 9, 12, 22, 24, 14, 21, 26, 0, 12, 2, 10, 24, 14, 21, 7, 17, 0, 12, 9, 22, 24, 14, 21, 24, 14, 19, 12, 17, 7, 17, 17, 19, 15, 18, 5, 20, 26, 0, 12, 24, 14, 21, 5, 20, 8, 26, 0, 12, 16, 14, 21, 6, 19, 0, 12, 16, 14, 21, 20, 26, 0, 12, 24, 14, 21, 24, 14, 21, 6, 9, 11, 2, 12, 22, 24, 14, 19, 12, 17, 19, 12, 22, 24, 14, 19, 15, 18, 5, 20, 26, 0, 12, 2, 12, 22, 2, 12, 2, 12, 9, 22, 24, 14, 21, 6, 16, 14, 21, 24, 14, 21, 24, 14, 21, 6, 2, 12, 22, 24, 14, 21, 16, 14, 21, 2, 12, 9, 22, 24, 14, 21, 6, 12, 22, 24, 14, 19, 0, 12, 9, 11, 2, 12, 22, 24, 14, 21, 20, 26, 0, 12, 9, 11, 2, 12, 24, 14, 21, 20, 8, 

In [52]:
model.tag_to_ix

{'O': 0,
 'B-DRUG': 1,
 'I-DRUG': 2,
 'B-DRUG_INGREDIENT': 3,
 'I-DRUG_INGREDIENT': 4,
 'B-DISEASE': 5,
 'I-DISEASE': 6,
 'B-SYMPTOM': 7,
 'I-SYMPTOM': 8,
 'B-SYNDROME': 9,
 'I-SYNDROME': 10,
 'B-DISEASE_GROUP': 11,
 'I-DISEASE_GROUP': 12,
 'B-FOOD': 13,
 'I-FOOD': 14,
 'B-FOOD_GROUP': 15,
 'I-FOOD_GROUP': 16,
 'B-PERSON_GROUP': 17,
 'I-PERSON_GROUP': 18,
 'B-DRUG_GROUP': 19,
 'I-DRUG_GROUP': 20,
 'B-DRUG_DOSAGE': 21,
 'I-DRUG_DOSAGE': 22,
 'B-DRUG_TASTE': 23,
 'I-DRUG_TASTE': 24,
 'B-DRUG_EFFICACY': 25,
 'I-DRUG_EFFICACY': 26,
 '<START>': 27,
 '<STOP>': 28}

In [84]:
feats[0]

tensor([[ 0.6181, -0.5840,  0.4561,  ..., -0.2970, -0.5666,  0.6928],
        [ 1.1452, -0.8345,  0.0617,  ..., -0.3726, -0.3890, -0.2906],
        [ 1.0879, -0.5812, -0.0476,  ..., -0.5300,  0.4309,  0.2381],
        ...,
        [ 0.3998,  0.1130, -0.0358,  ..., -0.1121,  0.6790, -0.4890],
        [ 0.0173, -0.1837,  0.4009,  ..., -0.1481,  0.1281, -1.4989],
        [ 0.9533,  0.0157,  0.0977,  ...,  0.3136,  0.5665, -0.5943]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [88]:
feats[0].index_select(0, (inputs['labels']!=-100)[0].nonzero().squeeze()).shape

torch.Size([153, 27])

In [85]:
(inputs['labels']!=-100)[0].nonzero().squeeze()

tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
         85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
         99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153],
       device='cuda:0')

In [79]:
(inputs['labels']!=-100).nonzero(as_tuple=True)

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2

In [59]:
inputs['labels']

tensor([[-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,   25,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100]], device='cuda:0')

In [75]:
feats

tensor([[[ 6.1807e-01, -5.8396e-01,  4.5611e-01,  ..., -2.9698e-01,
          -5.6658e-01,  6.9278e-01],
         [ 1.1452e+00, -8.3447e-01,  6.1680e-02,  ..., -3.7263e-01,
          -3.8899e-01, -2.9056e-01],
         [ 1.0879e+00, -5.8116e-01, -4.7578e-02,  ..., -5.2997e-01,
           4.3086e-01,  2.3806e-01],
         ...,
         [ 3.9981e-01,  1.1297e-01, -3.5812e-02,  ..., -1.1207e-01,
           6.7896e-01, -4.8900e-01],
         [ 1.7350e-02, -1.8369e-01,  4.0088e-01,  ..., -1.4807e-01,
           1.2806e-01, -1.4989e+00],
         [ 9.5327e-01,  1.5746e-02,  9.7731e-02,  ...,  3.1362e-01,
           5.6654e-01, -5.9433e-01]],

        [[ 2.0840e-01, -4.5599e-01,  6.1060e-01,  ..., -3.4155e-01,
          -3.1615e-02,  5.7254e-01],
         [ 2.4314e-01, -1.2979e+00, -6.1521e-01,  ...,  9.7411e-02,
          -2.5493e-03, -1.2358e+00],
         [ 5.5959e-01, -2.4002e-01,  2.6309e-01,  ...,  3.0448e-01,
           8.3479e-01, -8.8549e-02],
         ...,
         [ 9.4868e-01, -2

In [74]:
torch.where(t != -100, feats, torch.tensor(-1.).to('cuda'))

tensor([[[-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
          -1.0000e+00, -1.0000e+00],
         [ 1.1452e+00, -8.3447e-01,  6.1680e-02,  ..., -3.7263e-01,
          -3.8899e-01, -2.9056e-01],
         [ 1.0879e+00, -5.8116e-01, -4.7578e-02,  ..., -5.2997e-01,
           4.3086e-01,  2.3806e-01],
         ...,
         [-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
          -1.0000e+00, -1.0000e+00],
         [-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
          -1.0000e+00, -1.0000e+00],
         [-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
          -1.0000e+00, -1.0000e+00]],

        [[-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
          -1.0000e+00, -1.0000e+00],
         [ 2.4314e-01, -1.2979e+00, -6.1521e-01,  ...,  9.7411e-02,
          -2.5493e-03, -1.2358e+00],
         [ 5.5959e-01, -2.4002e-01,  2.6309e-01,  ...,  3.0448e-01,
           8.3479e-01, -8.8549e-02],
         ...,
         [-1.0000e+00, -1

In [76]:
torch.nonzero(t != -100)

<ipython-input-76-10624ec25d77>:1: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  torch.nonzero(t != -100)


tensor([[ 0,  1,  0],
        [ 0,  2,  0],
        [ 0,  3,  0],
        ...,
        [ 5, 78,  0],
        [ 5, 79,  0],
        [ 5, 80,  0]], device='cuda:0')

In [68]:
t != -100

tensor([[[False],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]]], device='cuda:0')

In [67]:
t[t != -100]

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  8,  8,  8,  9, 10, 10, 10,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 12, 12,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 25, 26, 26, 26,  0, 25, 26, 26, 26,  0,  0,  0,  7,
         8,  8,  8,  8,  8,  8,  8,  0,  7,  8,  8,  8,  0,  0,  0,  0,  0,  0,
         0,  0,  5,  6,  6,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 

In [63]:
t = inputs['labels'].unsqueeze(-1)
t

tensor([[[-100],
         [   0],
         [   0],
         ...,
         [-100],
         [-100],
         [-100]],

        [[-100],
         [   0],
         [   0],
         ...,
         [-100],
         [-100],
         [-100]],

        [[-100],
         [   0],
         [   0],
         ...,
         [-100],
         [-100],
         [-100]],

        [[-100],
         [   0],
         [   0],
         ...,
         [-100],
         [-100],
         [-100]],

        [[-100],
         [   0],
         [  25],
         ...,
         [-100],
         [-100],
         [-100]],

        [[-100],
         [   0],
         [   0],
         ...,
         [-100],
         [-100],
         [-100]]], device='cuda:0')

In [55]:
feats

tensor([[[ 6.1807e-01, -5.8396e-01,  4.5611e-01,  ..., -2.9698e-01,
          -5.6658e-01,  6.9278e-01],
         [ 1.1452e+00, -8.3447e-01,  6.1680e-02,  ..., -3.7263e-01,
          -3.8899e-01, -2.9056e-01],
         [ 1.0879e+00, -5.8116e-01, -4.7578e-02,  ..., -5.2997e-01,
           4.3086e-01,  2.3806e-01],
         ...,
         [ 3.9981e-01,  1.1297e-01, -3.5812e-02,  ..., -1.1207e-01,
           6.7896e-01, -4.8900e-01],
         [ 1.7350e-02, -1.8369e-01,  4.0088e-01,  ..., -1.4807e-01,
           1.2806e-01, -1.4989e+00],
         [ 9.5327e-01,  1.5746e-02,  9.7731e-02,  ...,  3.1362e-01,
           5.6654e-01, -5.9433e-01]],

        [[ 2.0840e-01, -4.5599e-01,  6.1060e-01,  ..., -3.4155e-01,
          -3.1615e-02,  5.7254e-01],
         [ 2.4314e-01, -1.2979e+00, -6.1521e-01,  ...,  9.7411e-02,
          -2.5493e-03, -1.2358e+00],
         [ 5.5959e-01, -2.4002e-01,  2.6309e-01,  ...,  3.0448e-01,
           8.3479e-01, -8.8549e-02],
         ...,
         [ 9.4868e-01, -2

In [45]:
outputs.logits.shape

torch.Size([6, 254, 27])

In [46]:
len(outputs.hidden_states)

13

In [44]:
outputs.hidden_states

(tensor([[[ 0.2114,  0.0041, -0.1732,  ..., -0.3671,  0.0000, -0.1438],
          [-0.7619,  0.5141,  0.0846,  ..., -0.4025,  0.3214,  1.2050],
          [-0.2144, -1.1778, -1.4555,  ..., -0.3486, -0.2206,  0.7280],
          ...,
          [ 0.5109,  0.0000,  0.0000,  ...,  0.0000,  0.2027, -0.0705],
          [ 0.7674,  0.3430, -1.1291,  ...,  0.0000,  0.8625,  0.0000],
          [ 0.6818, -0.1219, -0.7438,  ...,  0.2688,  0.9632,  0.1991]],
 
         [[ 0.0000,  0.0041, -0.1732,  ..., -0.3671, -0.2131, -0.1438],
          [-0.7619,  0.5141,  0.0846,  ..., -0.4025,  0.3214,  1.2050],
          [-0.8276,  0.3144,  0.0000,  ...,  0.2720,  1.0127,  0.3822],
          ...,
          [ 0.5109,  0.0000, -0.5653,  ...,  0.4510,  0.0000, -0.0705],
          [ 0.0000,  0.0000, -1.1291,  ...,  0.0000,  0.8625,  0.0256],
          [ 0.6818, -0.1219, -0.7438,  ...,  0.0000,  0.9632,  0.0000]],
 
         [[ 0.0000,  0.0000, -0.1732,  ..., -0.3671, -0.2131, -0.1438],
          [-1.7798, -0.3042,

In [51]:
p = EvalPrediction(predictions=preds_, label_ids=label_idst)
preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)

### decode

In [52]:
from glob import glob

In [39]:
import torch
torch.cuda.set_device(0)

In [41]:
if torch.cuda:
    torch.cuda.set_device('cuda:0')

In [46]:
model = MyBertForTokenClassification.from_pretrained('prerained-0-0', config=config)


In [48]:
model.config.num_labels

27

In [43]:
preds_ = None
with torch.no_grad():
    for fp in glob('./param*'):
#         print(fp)
#         model = MyBertForTokenClassification.from_pretrained('prerained-0-0', config=config)

#         model.load_state_dict(torch.load(fp))
#         model = model.to('cuda')
#         model.eval()
        preds: torch.Tensor = None
        label_ids: torch.Tensor = None
        for stepv, inputsv in enumerate(test_data_loader):
#             del inputsv['labels']
            for k, v in inputsv.items():
                if isinstance(v, torch.Tensor):
                    inputsv[k] = v.to('cuda')
            outputs = model(**inputsv)
            logits = outputs[1]
            preds = logits if preds is None else nested_concat(preds, logits, dim=0)
            label_ids = inputsv['labels'] if label_ids is None else nested_concat(label_ids, inputsv['labels'], dim=0)
        preds = nested_numpify(preds)
        preds_ = preds if preds_ is None else preds_+preds
        label_ids = nested_numpify(label_ids)
        break
    p = EvalPrediction(predictions=preds_, label_ids=label_ids)
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    #     break

In [53]:
!mkdir {args.DATA_GEN}sub_pytorch5

In [54]:
result = []
pre_guid = None
ti = 1
base_len = 0
for i in range(len(preds_list)):
#     print(i)
    guid = examples_test[i].guid.split('-')[0]
#     print(guid)
    if guid != pre_guid:
        if pre_guid:
            with open(f'{args.DATA_GEN}sub_pytorch5/{pre_guid}.ann', 'w', encoding='utf8') as f:
                f.write('\n'.join(result))
            result = []
            
            ti = 1
            base_len = 0
    else:
        print(guid)
    cat = None
    start, end = None, None
    
    for ii in range(len(preds_list[i])):
        if preds_list[i][ii].startswith('B'):
            cat = preds_list[i][ii].split('-')[-1]
            start = ii
        elif preds_list[i][ii] == 'O':
            if cat is not None and start is not None:
                end = ii
                result.append(f'T{ti}\t{cat} {start+base_len} {end+base_len}\t{"".join(examples_test[i].words[start: end])}')
                ti += 1
                cat = None
                start, end = None, None
    pre_guid = guid
    base_len += len(examples_test[i].words)
if pre_guid:
    with open(f'{args.DATA_GEN}sub_pytorch5/{pre_guid}.ann', 'w', encoding='utf8') as f:
        f.write('\n'.join(result))

1232
1232
1264
1103
1103
1103
1103
1071
1192
1192
1183
1190
1190
1166
1331
1045
1175
1175
1175
1202
1014
1014
1203
1359
1359
1359
1022
1182
1182
1197
1456
1456
1387
1387
1308
1016
1496
1128
1239
1026
1445
1237
1168
1168
1168
1215
1215
1215
1225
1225
1227
1257
1350
1253
1151
1151
1151
1212
1212
1212
1212
1338
1069
1002
1002
1362
1346
1236
1282
1283
1400
1400
1400
1420
1420
1420
1224
1224
1148
1148
1148
1222
1222
1222
1222
1410
1410
1410
1072
1191
1191
1090
1310
1470
1470
1470
1251
1251
1251
1251
1251
1438
1419
1419
1226
1406
1406
1047
1467
1141
1160
1311
1132
1150
1206
1340
1340
1340
1234
1043
1067
1180
1049
1401
1401
1488
1488
1488
1216
1216
1216
1116
1296
1296
1498
1076
1377
1003
1363
1363
1010
1010
1010
1162
1223
1321
1254
1015
1122
1301
1061
1126
1319
1319
1124
1124
1270
1270
1270
1198
1315
1248
1248
1248
1127
1127
1127
1382
1382
1473
1473
1453
1453
1110
1110
1110
1323
1155
1402
1402
1121
1034
1034
1131
1105
1057
1057
1170
1144
1044
1474
1474
1479
1281
1138
1138
1077
1255
1255
1255


In [37]:
len(preds_list)

851

In [102]:
import re

In [104]:
len(re.sub(r'\s+', '', ''.join(examples_test[0].words)))

112

In [105]:
len(re.sub(r'\s+', '', ''.join(examples_test[-3].words)))

69

In [106]:
len(re.sub(r'\s+', '', ''.join(examples_test[2].words)))

221

In [116]:
len(examples_test[0].words), len(preds_list[0])

(125, 125)

In [119]:
len(examples_test[2].words), len(preds_list[2])

(240, 240)

In [120]:
len(examples_test[21].words), len(preds_list[21])

(146, 146)

In [117]:
examples_test[0].words

[' ',
 '孕',
 '妇',
 '忌',
 '服',
 '。',
 ' ',
 ' ',
 '养',
 '血',
 '化',
 '瘀',
 '，',
 '舒',
 '肝',
 '调',
 '经',
 '。',
 '用',
 '于',
 '气',
 '滞',
 '血',
 '瘀',
 '所',
 '致',
 '的',
 '月',
 '经',
 '不',
 '调',
 '，',
 '痛',
 '经',
 '，',
 '症',
 '见',
 '月',
 '经',
 '量',
 '少',
 '有',
 '血',
 '块',
 '、',
 '经',
 '行',
 '后',
 '期',
 '或',
 '前',
 '后',
 '不',
 '定',
 '、',
 '经',
 '行',
 '小',
 '腹',
 '胀',
 '痛',
 '孕',
 '妇',
 ' ',
 '非',
 '处',
 '方',
 '药',
 '物',
 '（',
 '甲',
 '类',
 '）',
 ' ',
 ' ',
 '口',
 '服',
 '，',
 '一',
 '次',
 '1',
 '丸',
 '，',
 '一',
 '日',
 '2',
 '次',
 '。',
 ' ',
 ' ',
 '每',
 '丸',
 '重',
 '9',
 'g',
 ' ',
 ' ',
 '天',
 '津',
 '中',
 '新',
 '药',
 '业',
 '集',
 '团',
 '股',
 '份',
 '有',
 '限',
 '公',
 '司',
 '达',
 '仁',
 '堂',
 '制',
 '药',
 '厂',
 ' ',
 ' ',
 '孕',
 '妇',
 '忌',
 '服',
 '。',
 ' ']

In [118]:
preds_list[0]

['O',
 'B-PERSON_GROUP',
 'I-PERSON_GROUP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'O',
 'B-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'O',
 'O',
 'O',
 'B-SYNDROME',
 'I-SYNDROME',
 'I-SYNDROME',
 'I-SYNDROME',
 'O',
 'O',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'O',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'O',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'B-PERSON_GROUP',
 'I-PERSON_GROUP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [ ]:
test_data_loader

In [31]:
len(preds_list),len(out_label_list)

(392, 392)

In [33]:
len(preds_list[0]),len(out_label_list[0])

(141, 141)

In [34]:
len(preds_list[1]),len(out_label_list[1])

(230, 230)

In [32]:
out_label_list[0]

['O',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'I-DRUG_EFFICACY',
 'O',
 'O',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'I-SYMPTOM',
 'O',
 'B-SYMPTOM',
 'I-

In [28]:
preds_list

[['O',
  'O',
  'B-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'O',
  'B-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'O',
  'B-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'O',
  'B-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DRUG_EFFICACY',
  'I-DRUG_EFFICACY',
  'I-DRUG_EFFICACY',
  'I-DRUG_EFFICACY',
  'O',
  'B-DRUG_EFFICACY',
  'I-DRUG_EFFICACY',
  'I-DRUG_EFFICACY',
  'I-DRUG_EFFICACY',
  'O',
  'O',
  'O',
  'B-SYMPTOM',
  'I-SYMPTOM',
  'I-SYMPTOM',
 

In [23]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(examples)):
#     print(trn_idx, val_idx)
    examples_train = pd.Series(examples).iloc[trn_idx].tolist()
    examples_val = pd.Series(examples).iloc[val_idx].tolist()
    trainset = NerDataset(examples_train, tokenizer, labels, config.model_type, max_seq_length)
    train_data_loader = DataLoader(trainset, batch_size=6, shuffle=True,collate_fn=default_data_collator)
    valset = NerDataset(examples_val, tokenizer, labels, config.model_type, max_seq_length)
    val_data_loader = DataLoader(valset, batch_size=6, shuffle=True,collate_fn=default_data_collator)
    model = MyBertForTokenClassification.from_pretrained(model_name_or_path, config=config)
    
    model = model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    
    time_start = datetime.now()
    
    epochs = 50
    for epoch in range(epochs):
        losst = 0
        model.train()
        for step, inputs in enumerate(train_data_loader):
            optimizer.zero_grad()
        #     print(step)
        #     print(inputs)
            for k, v in inputs.items():
                if isinstance(v, torch.Tensor):
                    inputs[k] = v.to('cuda')
#             print(inputs)
#             break
            outputs = model(**inputs)
            loss = outputs[0]
            losst += loss.item()
            loss.backward()
            optimizer.step()
#             if step % 100 == 0:
#                 print(epoch, step, loss.item())
        
        with torch.no_grad():
            model.eval()
            lossv = 0
            preds: torch.Tensor = None
            label_ids: torch.Tensor = None
            for stepv, inputsv in enumerate(val_data_loader):
                for k, v in inputsv.items():
                    if isinstance(v, torch.Tensor):
                        inputsv[k] = v.to('cuda')
                outputs = model(**inputsv)
                lossv += outputs[0] 
                logits = outputs[1]
                preds = logits if preds is None else nested_concat(preds, logits, dim=0)
                label_ids = inputsv['labels'] if label_ids is None else nested_concat(label_ids, inputsv['labels'], dim=0)
            preds = nested_numpify(preds)
            label_ids = nested_numpify(label_ids)
            metrics = compute_metrics(EvalPrediction(predictions=preds, label_ids=label_ids))
            now = datetime.now()
            print(f'{now.strftime("%Y-%m-%d %H:%M:%S")} {int((now-time_start).total_seconds())}s epoch: {epoch}; train loss: {losst}; val loss: {lossv}\nmetrics:{metrics}')
        
        time_start = now
    break

2020-09-30 16:58:08.923 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d
2020-09-30 16:58:13.903 | INFO     | __main__:convert_examples_to_features:31 - Writing example %d of %d
Some weights of the model checkpoint at bert-base-cased were not used when initializing MyBertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing MyBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing MyBertForTokenClassification from the checkpoint of a model that you expect to be exact

2020-09-30 16:58:53 35s epoch: 0; train loss: 233.0487336218357; val loss: 46.893863677978516
metrics:{'precision': 0.15670969678775143, 'recall': 0.14560669456066946, 'f1': 0.15095430884904568}
2020-09-30 16:59:28 35s epoch: 1; train loss: 166.47629860043526; val loss: 39.51923751831055
metrics:{'precision': 0.2821650399290151, 'recall': 0.26610878661087867, 'f1': 0.2739018087855297}
2020-09-30 17:00:04 35s epoch: 2; train loss: 138.5117626786232; val loss: 33.81781768798828
metrics:{'precision': 0.33513206475433116, 'recall': 0.3291492329149233, 'f1': 0.3321137067267098}
2020-09-30 17:00:39 35s epoch: 3; train loss: 124.96450391411781; val loss: 32.58758544921875
metrics:{'precision': 0.3378901212277534, 'recall': 0.36541143654114366, 'f1': 0.35111230233181456}
2020-09-30 17:01:15 35s epoch: 4; train loss: 115.32277190685272; val loss: 31.041973114013672
metrics:{'precision': 0.3672627235213205, 'recall': 0.3723849372384937, 'f1': 0.3698060941828255}
2020-09-30 17:01:51 35s epoch: 5;

KeyboardInterrupt: 

In [27]:
for stepv, inputsv in enumerate(val_data_loader):
    for k, v in inputsv.items():
        if isinstance(v, torch.Tensor):
            inputsv[k] = v.to('cuda')
    outputs = model(**inputsv)
    break

In [28]:
outputs[0]

tensor(0.7820, device='cuda:0', grad_fn=<NllLossBackward>)

In [33]:
inputsv['labels'].size()

torch.Size([6, 250])

In [30]:
outputs[1].size()

torch.Size([6, 250, 27])

In [60]:
outputs[1]

tensor([[[ 7.2478, -0.9541,  0.3502,  ..., -1.7577,  0.8842,  1.6488],
         [ 9.1770,  0.4352,  1.2137,  ..., -1.2776,  1.2094, -1.8445],
         [ 9.2644,  1.6467,  1.2957,  ..., -2.3641, -0.5144,  0.7105],
         ...,
         [ 7.5256, -0.1242,  0.6146,  ..., -1.9995, -0.8282,  0.6976],
         [ 6.6889, -0.8332,  0.5084,  ..., -1.7485, -0.5956,  0.9563],
         [ 7.7120,  0.1404,  0.7082,  ..., -0.8425, -0.8540,  0.2866]],

        [[ 7.6796,  0.0823,  0.4245,  ..., -1.7960,  0.1496,  0.8459],
         [10.2698,  0.8696,  1.0195,  ..., -1.0480,  1.3240, -1.1397],
         [ 8.5301,  1.4456,  0.8132,  ..., -2.6058, -1.2604,  1.3601],
         ...,
         [ 8.3323, -0.8301,  0.1058,  ..., -1.7747, -0.8403, -0.2943],
         [ 6.8183, -0.8213,  0.8348,  ..., -1.5840, -0.7093,  0.0195],
         [ 7.3432,  0.2935,  0.7771,  ..., -1.3801, -1.3548, -0.1197]],

        [[ 7.1986, -1.1691, -1.4432,  ..., -1.6779,  0.7274,  1.7633],
         [ 5.2698, -0.9619, -2.1792,  ..., -1

In [64]:
preds = nested_numpify(outputs[1].detach())

In [65]:
label_ids = nested_numpify(inputsv['labels'])

In [83]:
metrics = compute_metrics(EvalPrediction(predictions=preds, label_ids=label_ids))

229 229
229 229
81 81
217 217
214 214
88 88
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [74]:
metrics

{'precision': 0.0, 'recall': 0.0, 'f1': 0}

In [49]:
id2label= {i: label for i, label in enumerate(labels)}
label2id={label: i for i, label in enumerate(labels)}

In [50]:
y_true =inputsv['labels'].cpu().detach().numpy()

In [51]:
preds = np.argmax(outputs[1].cpu().detach().numpy(), 2)

In [52]:
preds.shape

(6, 250)

In [53]:
y_true.shape

(6, 250)